In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress
from config import apikey


In [ ]:
#2018-2019 season was from 10/16/2018 to 4/10/2019
# URL for GET requests to retrieve NBA game ids from the season url
scheduleurl = "https://api.sportradar.us/nba/trial/v7/en/games/2018/reg/schedule.json?api_key="
# Print the response object to the console
response = requests.get(scheduleurl+apikey).json()
print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
# The "response" property in articles contains the actual articles
# list comprehension.
gameids = response["games"]

In [ ]:
gameids_list = []
date = []
city = []
country = []
hometeam = []
awayteam = []

# Print the web_url of each stored article
for gameid in gameids:
    gameids_list.append(gameid["id"])
    date.append(gameid["scheduled"])
    city.append(gameid["venue"]["city"])
    country.append(gameid["venue"]["country"])
    hometeam.append(gameid["home"]["name"])
    awayteam.append(gameid["away"]["name"])
    

In [ ]:
NBA_df = pd.DataFrame({
    "GameID": gameids_list,
    "Date": date,
    "City":city,
    "Country":country,
    "Home Team":hometeam,
    "Away Team":awayteam
})
NBA_df

In [ ]:
#do it again for 2019
#2019-2020 season was from 10/22/2019 to 3/11/2020
# URL for GET requests to retrieve NBA game ids from the season url
scheduleurl = "https://api.sportradar.us/nba/trial/v7/en/games/2019/reg/schedule.json?api_key="
# Print the response object to the console
response = requests.get(scheduleurl+apikey).json()
print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
# The "response" property in articles contains the actual articles
# list comprehension.
gameids = response["games"]

In [ ]:
gameids_list = []
date = []
city = []
country = []
hometeam = []
awayteam = []

# Print the web_url of each stored article
for gameid in gameids:
    gameids_list.append(gameid["id"])
    date.append(gameid["scheduled"])
    city.append(gameid["venue"]["city"])
    country.append(gameid["venue"]["country"])
    hometeam.append(gameid["home"]["name"])
    awayteam.append(gameid["away"]["name"])

In [ ]:
NBA19_df = pd.DataFrame({
    "GameID": gameids_list,
    "Date": date,
    "City":city,
    "Country":country,
    "Home Team":hometeam,
    "Away Team":awayteam
    })
NBA19_df

In [ ]:
NBAALL_df = NBA_df.append(NBA19_df)
NBAALL_df

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
NBAALL_df.to_csv("Output/NBAALL_df.csv", index=False, header=True)

In [ ]:
# Now we have the all games id
# Starting the get game info from game id

In [ ]:
NBAschedule = "Output/NBA900.csv"
# Not every CSV requires an encoding, but be aware this can come up
NBAschedule_df = pd.read_csv(NBAschedule, encoding="ISO-8859-1")
# Show just the header
NBAschedule_df.head()

In [ ]:
gameids_list=NBAschedule_df["ï»¿GameID"]
print(gameids_list)

In [ ]:
# tv maze show search base url
boxscoreurl = "http://api.sportradar.us/nba/trial/v7/en/games/"
endboxscoreurl = "/summary.json?api_key="+apikey

# set up lists to hold response data for name and rating
#main tree
attendances = []
#home then points
homepointstotals = []
#scoring, 0, points
home1stquarterpoints = []
#home then statistics
home_field_goals_pcts = []
home_three_points_pcts = []
home_free_throws_pcts = []
#away then points
awaypointstotals = []
#scoring, 0, points
away1stquarterpoints = []
#away then statistics
away_field_goals_pcts = []
away_three_points_pcts = []
away_free_throws_pcts = []


In [ ]:
# loop through game ids, make requests and parse and append
for gameids in gameids_list:
    target_url = boxscoreurl + gameids + endboxscoreurl
    boxscore_response = requests.get(target_url)
    time.sleep(1)
    boxscore_response_json = boxscore_response.json()
# Get attendance from JSON response
    # Try to grab the attendance figure from the json if not move on.  
    try:
        print(f"{gameids} found! Appending stats")
        attendances.append(boxscore_response_json['attendance'])
        homepointstotals.append(boxscore_response_json['home']['points'])
        home1stquarterpoints.append(boxscore_response_json['home']['scoring'][0]['points'])
        home_field_goals_pcts.append(boxscore_response_json['home']['statistics']['field_goals_pct'])
        home_three_points_pcts.append(boxscore_response_json['home']['statistics']['three_points_pct'])
        home_free_throws_pcts.append(boxscore_response_json['home']['statistics']['free_throws_pct'])
        awaypointstotals.append(boxscore_response_json['away']['points'])
        away1stquarterpoints.append(boxscore_response_json['away']['scoring'][0]['points'])
        away_field_goals_pcts.append(boxscore_response_json['away']['statistics']['field_goals_pct'])
        away_three_points_pcts.append(boxscore_response_json['away']['statistics']['three_points_pct'])
        away_free_throws_pcts.append(boxscore_response_json['away']['statistics']['free_throws_pct'])
        
    # Handle exceptions for a character that is not available in the Star Wars API
    except:
        # Append null values
        print("Character not found")
        pass  

In [ ]:
NBAfull_df = pd.DataFrame({
    "attendance": attendances,
    "Home Score": homepointstotals,
    "Home 1st Q":home1stquarterpoints,
    "Home FG%":home_field_goals_pcts,
    "Home 3pt%":home_three_points_pcts,
    "Home FT%":home_free_throws_pcts,
    "Away Score":awaypointstotals,
    "Away 1st Q":away1stquarterpoints,
    "Away FG%":away_field_goals_pcts,
    "Away 3pt%":away_three_points_pcts,
    "Away FT%":away_free_throws_pcts
})
NBAfull_df

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
NBAfull_df.to_csv("Output/NBAfull_df.csv", index=False, header=True)


In [ ]:
# Now we have games info for each games
# Starting data clean and try to make one csv file for all information
# NBAALL_df.csv shows season 2018 and 2019 all game id, Date, City, Country, Home Team, Away Team
# NBAfull_df.csv shows game information; attendance, Home Score, Home 1st Q, Home FG%,Home 3pt%,Home FT%, Away Score,Away 1st Q, Away FG%,Away 3pt%, Away FT%
# NbaData.csv our clean cvs file and It shows everything